In [ ]:
import pandas as pd

# Load the annotations
df = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')

# Display the first few rows of the dataframe
df.head()


,name,x1,x2,y1,y2,classname
0,2756.png,69,126,294,392,face_with_mask
1,2756.png,505,10,723,283,face_with_mask
2,2756.png,75,252,264,390,mask_colorful
3,2756.png,521,136,711,277,mask_colorful
4,6098.jpg,360,85,728,653,face_no_mask


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2

# Define image size and batch size
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Function to preprocess the data
def preprocess_data(df, img_dir):
    images = []
    labels = []
    for index, row in df.iterrows():
        img_path = os.path.join(img_dir, row['name'])

        # Check if image file exists
        if not os.path.exists(img_path):
            print(f"Image file not found: {img_path}")
            continue

        image = load_img(img_path)
        image = img_to_array(image)
        x1, x2, y1, y2 = row['x1'], row['x2'], row['y1'], row['y2']

        # Ensure bounding box coordinates are within the image bounds
        height, width = image.shape[:2]
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(width, x2)
        y2 = min(height, y2)

        face = image[y1:y2, x1:x2]

        # Check if face region is valid
        if face.size == 0:
            print(f"Skipping empty face region for image: {row['name']} at index: {index}")
            continue

        face = cv2.resize(face, IMG_SIZE) / 255.0
        images.append(face)
        labels.append(1 if row['classname'] == 'with_mask' else 0)

    return np.array(images), np.array(labels)

# Directory where images are stored
img_dir = '/content/drive/MyDrive/dataset/Medical mask/Medical mask/Medical Mask/images'

# Load your CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')
# Load and preprocess the data
X, y = preprocess_data(df, img_dir)

# Check if any images were skipped
print(f"Total images processed: {len(X)}")
print(f"Total labels processed: {len(y)}")

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")


Streaming output truncated to the last 5000 lines.
Skipping empty face region for image: 3002.png at index: 9360
Skipping empty face region for image: 3002.png at index: 9361
Skipping empty face region for image: 3002.png at index: 9364
Skipping empty face region for image: 3002.png at index: 9365
Skipping empty face region for image: 3002.png at index: 9367
Skipping empty face region for image: 3002.png at index: 9370
Skipping empty face region for image: 3774.png at index: 9371
Skipping empty face region for image: 3774.png at index: 9372
Skipping empty face region for image: 3106.png at index: 9373
Skipping empty face region for image: 3106.png at index: 9374
Skipping empty face region for image: 5229.jpg at index: 9375
Skipping empty face region for image: 5229.jpg at index: 9376
Skipping empty face region for image: 5229.jpg at index: 9377
Skipping empty face region for image: 5229.jpg at index: 9378
Skipping empty face region for image: 2984.png at index: 9379
Skipping empty face

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use sigmoid for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    validation_data=(X_val, y_val),
                    epochs=50)


Epoch 1/50
69/69 [==============================] - 84s 1s/step - loss: 0.0114 - accuracy: 0.9872 - val_loss: 1.3945e-18 - val_accuracy: 1.0000
Epoch 2/50
69/69 [==============================] - 82s 1s/step - loss: 4.0732e-15 - accuracy: 1.0000 - val_loss: 1.2338e-18 - val_accuracy: 1.0000
Epoch 3/50
69/69 [==============================] - 82s 1s/step - loss: 1.3246e-12 - accuracy: 1.0000 - val_loss: 1.2337e-18 - val_accuracy: 1.0000
Epoch 4/50
69/69 [==============================] - 81s 1s/step - loss: 1.3267e-15 - accuracy: 1.0000 - val_loss: 1.2337e-18 - val_accuracy: 1.0000
Epoch 5/50
69/69 [==============================] - 79s 1s/step - loss: 1.7825e-18 - accuracy: 1.0000 - val_loss: 1.2337e-18 - val_accuracy: 1.0000
Epoch 6/50
69/69 [==============================] - 79s 1s/step - loss: 9.8894e-16 - accuracy: 1.0000 - val_loss: 1.2337e-18 - val_accuracy: 1.0000
Epoch 7/50
69/69 [==============================] - 78s 1s/step - loss: 9.8386e-15 - accuracy: 1.0000 - val_loss: 1.

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


18/18 [==============================] - 5s 285ms/step - loss: 1.2337e-18 - accuracy: 1.0000
Validation Loss: 1.2337216856493646e-18
Validation Accuracy: 1.0


In [ ]:
import os
import tensorflow as tf

# Assuming you have your model architecture defined in a function build_model()
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

# Build and compile the model
model = build_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Save the weights
save_dir = '/content/drive/MyDrive/dataset/'
weights_path = os.path.join(save_dir, 'model_weights.h5')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
model.save_weights(weights_path)

print(f"Model weights saved to {weights_path}")


Model weights saved to /content/drive/MyDrive/dataset/model_weights.h5


In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np

# Ensure the save directory exists
save_dir = '/content/drive/MyDrive/dataset/'
weights_path = os.path.join(save_dir, 'model_weights.h5')

# Define the model architecture
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

# Build and compile the model
model = build_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load the weights
model.load_weights(weights_path)

# Load the Haar cascade for face detection
haar_cascade_path = '/content/drive/MyDrive/dataset/haarcascade_frontalface_default.xml'

if not os.path.exists(haar_cascade_path):
    raise FileNotFoundError(f"Haar cascade file not found at {haar_cascade_path}")

face_cascade = cv2.CascadeClassifier(haar_cascade_path)

if face_cascade.empty():
    raise IOError("Failed to load Haar cascade file. Please check the path and the file's existence.")

# Function to preprocess frame for model input
def preprocess_frame(frame):
    face = cv2.resize(frame, (224, 224))
    face = np.expand_dims(face, axis=0)  # Add batch dimension
    face = face / 255.0  # Normalize to [0, 1]
    return face

# Function to perform inference on a video file
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect faces in the frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            preprocessed_face = preprocess_frame(face)
            prediction = model.predict(preprocessed_face)
            label = 'Mask' if np.argmax(prediction) == 1 else 'No Mask'

            color = (0, 255, 0) if label == 'Mask' else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        out.write(frame)

    cap.release()
    out.release()

# Perform inference on sample videos
process_video('/content/drive/MyDrive/dataset/Test_video1.mp4', '/content/drive/MyDrive/dataset/output_video1.mp4')
process_video('/content/drive/MyDrive/dataset/Test_video2.mp4', '/content/drive/MyDrive/dataset/output_video2.mp4')
process_video('/content/drive/MyDrive/dataset/Test_video3.mp4', '/content/drive/MyDrive/dataset/output_video3.mp4')

print("Inference completed and output videos saved.")


1/1 [==============================] - 0s 92ms/step
Inference completed and output videos saved.
